### Spark

In [77]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.datastax.spark:spark-cassandra-connector_2.11:2.3.0 --conf spark.cassandra.connection.host=172.31.80.79,172.31.94.11,172.31.86.202 pyspark-shell'

In [78]:
from pyspark import SparkContext
#sc = SparkContext("local", "jupyter - query c")
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [170]:
def load_and_get_table_df(keys_space_name, table_name):
    table_df = sqlContext.read\
        .format("org.apache.spark.sql.cassandra")\
        .options(table=table_name, keyspace=keys_space_name)\
        .load()
    return table_df

queryctheme = load_and_get_table_df('gdelt', 'queryctheme')

In [80]:
queryctheme.show(5, False)

+-----------------------------------------+---------+--------+---+-----+-------------+-----------+----+
|theme                                    |nbarticle|date    |day|month|source       |tonemean   |year|
+-----------------------------------------+---------+--------+---+-----+-------------+-----------+----+
|TAX_WORLDLANGUAGES_HOCHUNKS              |1        |20190101|01 |0101 |kscj.com     |0.0        |2019|
|TAX_AIDGROUPS_EUROPEAID                  |3        |20190104|04 |0104 |reliefweb.int|-0.18344255|2019|
|TAX_AIDGROUPS_EUROPEAID                  |2        |20190121|21 |0121 |reliefweb.int|0.89331806 |2019|
|TAX_AIDGROUPS_EUROPEAID                  |1        |20190403|03 |0403 |reliefweb.int|2.4955437  |2019|
|TAX_TERROR_GROUP_SOMALI_NATIONAL_MOVEMENT|2        |20190508|08 |0508 |msn.com      |-0.3253796 |2019|
+-----------------------------------------+---------+--------+---+-----+-------------+-----------+----+
only showing top 5 rows



In [81]:
queryctheme.describe().show(5, True)

+-------+----------------+------------------+--------------------+-----------------+------------------+----------+-------------------+------+
|summary|           theme|         nbarticle|                date|              day|             month|    source|           tonemean|  year|
+-------+----------------+------------------+--------------------+-----------------+------------------+----------+-------------------+------+
|  count|          337826|            337826|              337826|           337826|            337826|    337826|             337826|337826|
|   mean|            null|134.25250276769697|2.0190490170469414E7|14.19130558334764|490.18717327855165|      null|-1.4537335533942588|2019.0|
| stddev|            null| 306.2307123723173|  363.53243662878424|9.254802254398976| 363.5297541766197|      null| 3.1808110322797645|   0.0|
|    min|ACT_FORCEPOSTURE|                 1|            20190101|               01|              0101|          |              -30.0|  2019|
|    m

In [ ]:
df = queryctheme.toPandas()
df.head(5)

### Python

In [84]:
import numpy as np
import pandas as pd

from bokeh.layouts import gridplot, row, column
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

In [85]:
output_notebook()

Loading BokehJS ...

In [86]:
!pip install cassandra-driver

In [168]:
from cassandra.cluster import Cluster

cluster = Cluster(['172.31.86.202'])  # provide contact points and port
session = cluster.connect('gdelt')
rows = session.execute('select * from queryctheme')

In [169]:
df1 = df.groupby("date").agg({"nbarticle": "sum"})
df1.columns

KeyError: 'nbarticle'

In [89]:
df1.head(5)

,nbarticle
date,
20190101,64272
20190102,184971
20190103,198497
20190104,220769
20190105,74636


In [165]:
p1 = figure(title="tdb1", plot_width=1200, plot_height=350)
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = "année 2019"
p1.yaxis.axis_label = "nb d'articles parus"
p1.line(df1.index, df1["nbarticle"], color="#A6CEE3")
show(p1)

In [166]:
df["new_month"] = df["month"].astype(str).str[:2]
data = df
data

KeyError: 'month'

In [163]:
data = data.groupby(["new_month", "source"]).sum()

KeyError: 'new_month'

In [117]:
michel = pd.pivot_table(data, values="nbarticle", index=["date"],columns=["source"], aggfunc=np.sum, fill_value=0)

In [148]:
denis = michel
denis

source,,10000birds.com,1005freshradio.ca,100scopenotes.com,1010wcsi.com,1010xl.com,1011now.com,1019online.com,1019thebeatfm.com,1019therock.com,...,znsbahamas.com,zocalo.com.mx,zodiakmalawi.com,zooborns.com,zoomorphic.net,zrock103.com,zululandobserver.co.za,zumi.ng,zwnews.com,date
date,,,,,,,,,,,,,,,,,,,,,
20190101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2019-01-01
20190102,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2019-01-02
20190103,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2019-01-03
20190104,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2019-01-04
20190105,0,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2019-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191227,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2019-12-27
20191228,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2019-12-28
20191229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2019-12-29


In [149]:
denis["date"] = pd.to_datetime(denis.index, format=("%Y%m%d"))
data = denis

years = list(data.index)
months = list(data.columns)

# reshape to 1D array or rates with a month and year for each row.
df = pd.DataFrame(data.stack(), columns=['rate']).reset_index()
denis

source,,10000birds.com,1005freshradio.ca,100scopenotes.com,1010wcsi.com,1010xl.com,1011now.com,1019online.com,1019thebeatfm.com,1019therock.com,...,znsbahamas.com,zocalo.com.mx,zodiakmalawi.com,zooborns.com,zoomorphic.net,zrock103.com,zululandobserver.co.za,zumi.ng,zwnews.com,date
date,,,,,,,,,,,,,,,,,,,,,
20190101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2019-01-01
20190102,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2019-01-02
20190103,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2019-01-03
20190104,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2019-01-04
20190105,0,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2019-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191227,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2019-12-27
20191228,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2019-12-28
20191229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2019-12-29


In [162]:
from math import pi
import pandas as pd

from bokeh.io import show
from bokeh.models import LinearColorMapper, BasicTicker, PrintfTickFormatter, ColorBar
from bokeh.plotting import figure

date = list(data.index)
article_sources = list(data.columns)

# reshape to 1D array or rates with a month and year for each row.
df = pd.DataFrame(data.stack(), columns=['rate']).reset_index()
df['rate'] = pd.to_numeric(df['rate'], errors='coerce')
df = df.dropna(subset=['rate'])
df['rate'] = df['rate'].astype('int')

# this is the colormap from the original NYTimes plot
colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette=colors, low=df.rate.min(), high=df.rate.max())

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title="US Unemployment ({0} - {1})".format(years[0], years[-1]),
           x_range=date, y_range=article_sources,
           x_axis_location="above", plot_width=900, plot_height=400,
           tools=TOOLS, toolbar_location='below',
           tooltips=[('date', '@date'), ('rate', '@rate%')])

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "5pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = pi / 3

p.rect(x="date", y="article_sources", width=1, height=1,source=df, fill_color={'field': 'rate', 'transform': mapper},line_color=None)

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="5pt",
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%d%%"),
                     label_standoff=6, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')

show(p)      # show the plot

ValueError: Unrecognized range input: '[20190101, 20190102, 20190103, 20190104, 20190105, 20190106, 20190107, 20190108, 20190109, 20190110, 20190111, 20190112, 20190113, 20190114, 20190115, 20190116, 20190117, 20190118, 20190119, 20190120, 20190121, 20190122, 20190123, 20190124, 20190125, 20190126, 20190127, 20190128, 20190129, 20190130, 20190131, 20190201, 20190202, 20190203, 20190204, 20190205, 20190206, 20190207, 20190208, 20190209, 20190210, 20190211, 20190212, 20190213, 20190214, 20190215, 20190216, 20190217, 20190218, 20190219, 20190220, 20190221, 20190222, 20190223, 20190224, 20190225, 20190226, 20190227, 20190228, 20190301, 20190302, 20190303, 20190304, 20190305, 20190306, 20190307, 20190308, 20190309, 20190310, 20190311, 20190312, 20190313, 20190314, 20190315, 20190316, 20190317, 20190318, 20190319, 20190320, 20190321, 20190322, 20190323, 20190324, 20190325, 20190326, 20190327, 20190328, 20190329, 20190330, 20190331, 20190401, 20190402, 20190403, 20190404, 20190405, 20190406, 

In [123]:
from math import pi
import pandas as pd

from bokeh.io import show
from bokeh.models import LinearColorMapper, BasicTicker, PrintfTickFormatter, ColorBar
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

#denis["date"] = denis["date"].astype(str)
#denis = denis.set_index("date")
#data.drop('Annual', axis=1, inplace=True)
#denis.columns.name = "source"


TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title="blabla", y_range=source, plot_width=900, plot_height=400, x_axis_type = "datetime",
            tools=TOOLS, toolbar_location='below',tooltips=[('date', '@Month @Year'),('rate', '@rate%')])
    
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "5pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = pi / 3

p.line(x="date", y="10000birds.com", width=1, height=1,source=source)

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="5pt",
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%d%%"),
                     label_standoff=6, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')

show(p)      # show the plot

AttributeError: 'ValueError' object has no attribute 'message'

In [ ]:
p = figure(x_range=df2["DATE"], ...)

In [104]:
date = denis.index.unique()

In [105]:
date

Int64Index([20190101, 20190102, 20190103, 20190104, 20190105, 20190106,
            20190107, 20190108, 20190109, 20190110,
            ...
            20191209, 20191210, 20191211, 20191212, 20191213, 20191214,
            20191215, 20191216, 20191217, 20191218],
           dtype='int64', name='date', length=352)